In [1]:
# https://www.relataly.com/crypto-market-cluster-analysis-using-affinity-propagation-python/8114/

In [2]:
# ! pip install cryptocmd
# ! pip install seaborn

In [3]:
from cryptocmd import CmcScraper
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
from sklearn import cluster, covariance, manifold
import requests
import json

In [4]:
NUMBER_OF_CRYPTOCURRENCIES = 500
TIMEOUT = 10 # seconds

In [13]:
def createBasicData():

    # get data
    url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit={NUMBER_OF_CRYPTOCURRENCIES}&sortBy=market_cap&sortType=desc&convert=USD&cryptoType=all&tagType=all&audited=false'
    response = requests.get(url, timeout = TIMEOUT)
    data = json.loads(response.text)
    basicData = pd.DataFrame(data['data']['cryptoCurrencyList'])

    # filter data
    basicData = basicData[
        (basicData['isActive'] == 1) & 
        (basicData['dateAdded'].apply(lambda x: int(x[:4])) < 2021) & 
        (basicData['lastUpdated'].apply(lambda x: int(x[:4])) > 2022) &
        (basicData['tags'].apply(lambda x: 'stablecoin' not in x)) &
        (basicData['marketPairCount'] > 5)
    ]

    # export data
    basicData.to_csv('./data/basic_data.csv')

    # return data
    return basicData

In [12]:
def importBasicData():

    # import data
    basicData = pd.read_csv('./data/basic_data.csv')
    
    # return data
    return basicData

In [ ]:
# To update data, delete the old file from the directory.

try:
    basicData = importBasicData()
except FileNotFoundError:
    basicData = createBasicData()